In [30]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# Connect DataBase

In [31]:
from langchain_community.utilities import SQLDatabase

# Define database connection parameters

db_user = "root"
db_password = "q1w2e3r4"
db_host = "localhost"
db_name = "llm_asst"

# 数据库连接
# db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

print(db.dialect)
print(db.get_usable_table_names())


mysql
['Memos', 'RecurringSchedules', 'Schedules', 'Users']


# Create Chains

## Step 1: Identify the intent of user's query

In [64]:
import os
import logging
import pandas as pd
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate


llm = OllamaLLM(model="llama3.1")

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

messages = []
intent_examples = pd.read_csv('intent_fewshot.csv')
# intent_examples = intent_examples.rename(columns={'prompt': 'input', 'intent': 'output'})
intent_fewshot = intent_examples.to_dict(orient='records')
print(intent_fewshot)

[{'query': 'Please create a new schedule for my dentist appointment on November 5th at 3 PM.', 'intent': 'Create Schedules'}, {'query': 'Set up a weekly team meeting every Monday at 10 AM.', 'intent': 'Create RecurringSchedules'}, {'query': 'Add a note to remind me to buy groceries.', 'intent': 'Create Memos'}, {'query': 'Remove my lunch appointment from the schedule for tomorrow.', 'intent': 'Delete Schedules'}, {'query': 'Cancel my weekly yoga class scheduled for Thursdays.', 'intent': 'Delete RecurringSchedules'}, {'query': 'Delete the memo about calling my friend.', 'intent': 'Delete Memos'}, {'query': 'Change my dentist appointment to November 12th at 4 PM.', 'intent': 'Update Schedules'}, {'query': 'Update my weekly team meeting to start at 11 AM instead of 10 AM.', 'intent': 'Update RecurringSchedules'}, {'query': 'Edit my grocery list memo to include eggs and milk.', 'intent': 'Update Memos'}, {'query': 'What appointments do I have scheduled for next week?', 'intent': 'Query Sc

In [65]:
op_types = ["Create", "Query", "Update", "Delete"]
tables = db.get_usable_table_names().remove("Users")

In [73]:
import re
# zero-shot prompt

zs_intent_template = """
Identify the intent behind the following user query.
Classify the intent into one of the operation types (create, delete, update, query) and one of the tables (Schedules, RecurringSchedules, Memos).
e.g., create Schedule
If no intent from the list is found, respond with: "Intent not found."
User Query: '{input}'
Response (format: operation type + table):
"""

messages = [("system", zs_intent_template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

# check zero-shot prompt by intent_examples
answers = []
for i, row in intent_examples.iterrows():
    response = chain.invoke({"input": row["query"]})
    # match one of the op_types and tables
    # match_op = re.search(r"create|delete|update|query", response, re.IGNORECASE)
    # match_table = re.search(r"Schedule(s)?|RecurringSchedule(s)?|Memo(s)?", response, re.IGNORECASE)
    answers.append(response)
intent_examples["predicted_output"] = answers
intent_examples

,query,intent,predicted_output,fs_answers,p3_answers,p4_answers,p5_answers
0,Please create a new schedule for my dentist ap...,Create Schedules,create Schedule,"Based on the user query, I would determine the...",Create Schedules,Create Schedules,Delete Schedules
1,Set up a weekly team meeting every Monday at 1...,Create RecurringSchedules,"To identify the intent behind the user query, ...","Based on the user query, I would classify it a...",Create RecurringSchedules,Create RecurringSchedules,Intent: Create RecurringSchedules
2,Add a note to remind me to buy groceries.,Create Memos,create Memos \n\nThe user wants to add a new r...,"Based on the user query, I would determine the...",Create Memos,Create Memos,Intent: Create Memos
3,Remove my lunch appointment from the schedule ...,Delete Schedules,Update \nSchedule,"Based on the user query, I would determine the...",Delete Schedules,Delete Schedules,Delete Schedules
4,Cancel my weekly yoga class scheduled for Thur...,Delete RecurringSchedules,"Based on the user query, I would classify the ...","Based on the user query ""Cancel my weekly yoga...",Delete RecurringSchedules,Create RecurringSchedules \n\n(Note: The inten...,Intent: Delete RecurringSchedules
5,Delete the memo about calling my friend.,Delete Memos,"update + Memos \n\nHowever, since it's asking ...","Based on the user query and possible intents, ...",Delete Memos,Delete Memos,Delete Memos.
6,Change my dentist appointment to November 12th...,Update Schedules,Create Schedule \n\nThe intent behind the user...,"Based on the user query, I would determine the...",Create Schedules,Create Schedules,Delete Schedules
7,Update my weekly team meeting to start at 11 A...,Update RecurringSchedules,update Schedules,Intent: update RecurringSchedules,Create Schedules,Create Schedules \n\n(This operation involves ...,Delete Schedules
8,Edit my grocery list memo to include eggs and ...,Update Memos,"Based on the user query, I identify the intent...","Based on the user query, the intent is:\n\nupd...",Update Memos,Update Memos,Intent: Update Memos
9,What appointments do I have scheduled for next...,Query Schedules,"Intent not found. \n\nHowever, based on the qu...","Based on the user query, the intent can be inf...",Query Schedules.,Query Schedules,Query RecurringSchedules


In [67]:
# few-shot prompt
fs_intent_template = """
Determine the operation type and table from the user queries below. Use only the following candidates: operation types (create, delete, update, query) and tables (Schedules, RecurringSchedules, Memos).
---
Example:

Query: 'Please remove my appointment from the schedule.'
Intent: delete Schedules
Query: 'Update the weekly team meeting to 11 AM.'
Intent: update RecurringSchedules
Query: 'Create a note for my presentation next week.'
Intent: create Memos
Query: 'Can you show my upcoming events?'
Intent: query Schedules

---
If no intent from the list is found, respond with: "Intent not found."
User Query: {input}
Intent:"

"""

messages = [("system", fs_intent_template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

# check zero-shot prompt by intent_examples
answers = []
for i, row in intent_examples.iterrows():
    response = chain.invoke({"input": row["query"]})
    answers.append(response)
intent_examples["fs_answers"] = answers
intent_examples


,query,intent,predicted_output,fs_answers
0,Please create a new schedule for my dentist ap...,Create Schedules,create Schedule,"Based on the user query, I would determine the..."
1,Set up a weekly team meeting every Monday at 1...,Create RecurringSchedules,The intent behind the user query is to create ...,"Based on the user query, I would classify it a..."
2,Add a note to remind me to buy groceries.,Create Memos,Create + Memos \n\n(Note: The user wants to cr...,"Based on the user query, I would determine the..."
3,Remove my lunch appointment from the schedule ...,Delete Schedules,update Schedules \n\nThe query is asking to mo...,"Based on the user query, I would determine the..."
4,Cancel my weekly yoga class scheduled for Thur...,Delete RecurringSchedules,update Schedules \n\n(assuming that cancelling...,"Based on the user query ""Cancel my weekly yoga..."
5,Delete the memo about calling my friend.,Delete Memos,update + Memos,"Based on the user query and possible intents, ..."
6,Change my dentist appointment to November 12th...,Update Schedules,"To identify the intent behind the user query, ...","Based on the user query, I would determine the..."
7,Update my weekly team meeting to start at 11 A...,Update RecurringSchedules,create Schedule \n\nThe intent is to modify th...,Intent: update RecurringSchedules
8,Edit my grocery list memo to include eggs and ...,Update Memos,"Based on the user query, I identify the intent...","Based on the user query, the intent is:\n\nupd..."
9,What appointments do I have scheduled for next...,Query Schedules,update Schedules,"Based on the user query, the intent can be inf..."


In [68]:
intent_list = ", ".join(intent_examples.intent.unique())

template = """You are a helpful assistant capable of detecting the intent behind a user's query.
The intent is a label that describes the user's goal or purpose in their query.
Reply one of the following: """+ intent_list + """
If no intent from the list is found, respond with: "Intent not found."
You are asked to provide a response to the following user query:
""" 

messages = [("system", template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

# check zero-shot prompt by intent_examples
answers = []
for i, row in intent_examples.iterrows():
    response = chain.invoke({"input": row["query"]})
    answers.append(response)
intent_examples["p3_answers"] = answers
intent_examples


,query,intent,predicted_output,fs_answers,p3_answers
0,Please create a new schedule for my dentist ap...,Create Schedules,create Schedule,"Based on the user query, I would determine the...",Create Schedules
1,Set up a weekly team meeting every Monday at 1...,Create RecurringSchedules,The intent behind the user query is to create ...,"Based on the user query, I would classify it a...",Create RecurringSchedules
2,Add a note to remind me to buy groceries.,Create Memos,Create + Memos \n\n(Note: The user wants to cr...,"Based on the user query, I would determine the...",Create Memos
3,Remove my lunch appointment from the schedule ...,Delete Schedules,update Schedules \n\nThe query is asking to mo...,"Based on the user query, I would determine the...",Delete Schedules
4,Cancel my weekly yoga class scheduled for Thur...,Delete RecurringSchedules,update Schedules \n\n(assuming that cancelling...,"Based on the user query ""Cancel my weekly yoga...",Delete RecurringSchedules
5,Delete the memo about calling my friend.,Delete Memos,update + Memos,"Based on the user query and possible intents, ...",Delete Memos
6,Change my dentist appointment to November 12th...,Update Schedules,"To identify the intent behind the user query, ...","Based on the user query, I would determine the...",Create Schedules
7,Update my weekly team meeting to start at 11 A...,Update RecurringSchedules,create Schedule \n\nThe intent is to modify th...,Intent: update RecurringSchedules,Create Schedules
8,Edit my grocery list memo to include eggs and ...,Update Memos,"Based on the user query, I identify the intent...","Based on the user query, the intent is:\n\nupd...",Update Memos
9,What appointments do I have scheduled for next...,Query Schedules,update Schedules,"Based on the user query, the intent can be inf...",Query Schedules.


In [69]:
print(template)

You are a helpful assistant capable of detecting the intent behind a user's query.
The intent is a label that describes the user's goal or purpose in their query.
Reply one of the following: Create Schedules, Create RecurringSchedules, Create Memos, Delete Schedules, Delete RecurringSchedules, Delete Memos, Update Schedules, Update RecurringSchedules, Update Memos, Query Schedules, Query RecurringSchedules, Query Memos, Intent not found
If no intent from the list is found, respond with: "Intent not found."
You are asked to provide a response to the following user query:



In [70]:
intent_list = ", ".join(intent_examples.intent.unique())

template = """You are a helpful assistant capable of detecting the intent behind a user's query.
The intent is a label that describes the user's purpose in their query, including the operation type and table, In the context of managing time and tasks
---
Explanation: 
'Schedules' are one-time events, 'RecurringSchedules' are events that repeat, and 'Memos' are something to be remembered without a specific time.
'Create' adds new entries, 'Delete' removes existing entries, 'Update' modifies existing entries, 'Query' retrieves existing entries without making any changes.

---
Reply one of the following: """+ intent_list + """
If no intent from the list is found, respond with: "Intent not found."

You are asked to provide a response to the following user query:
""" 

messages = [("system", template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

# check zero-shot prompt by intent_examples
answers = []
for i, row in intent_examples.iterrows():
    response = chain.invoke({"input": row["query"]})
    answers.append(response)
intent_examples["p4_answers"] = answers
intent_examples


,query,intent,predicted_output,fs_answers,p3_answers,p4_answers
0,Please create a new schedule for my dentist ap...,Create Schedules,create Schedule,"Based on the user query, I would determine the...",Create Schedules,Create Schedules
1,Set up a weekly team meeting every Monday at 1...,Create RecurringSchedules,The intent behind the user query is to create ...,"Based on the user query, I would classify it a...",Create RecurringSchedules,Create RecurringSchedules
2,Add a note to remind me to buy groceries.,Create Memos,Create + Memos \n\n(Note: The user wants to cr...,"Based on the user query, I would determine the...",Create Memos,Create Memos
3,Remove my lunch appointment from the schedule ...,Delete Schedules,update Schedules \n\nThe query is asking to mo...,"Based on the user query, I would determine the...",Delete Schedules,Delete Schedules
4,Cancel my weekly yoga class scheduled for Thur...,Delete RecurringSchedules,update Schedules \n\n(assuming that cancelling...,"Based on the user query ""Cancel my weekly yoga...",Delete RecurringSchedules,Create RecurringSchedules \n\n(Note: The inten...
5,Delete the memo about calling my friend.,Delete Memos,update + Memos,"Based on the user query and possible intents, ...",Delete Memos,Delete Memos
6,Change my dentist appointment to November 12th...,Update Schedules,"To identify the intent behind the user query, ...","Based on the user query, I would determine the...",Create Schedules,Create Schedules
7,Update my weekly team meeting to start at 11 A...,Update RecurringSchedules,create Schedule \n\nThe intent is to modify th...,Intent: update RecurringSchedules,Create Schedules,Create Schedules \n\n(This operation involves ...
8,Edit my grocery list memo to include eggs and ...,Update Memos,"Based on the user query, I identify the intent...","Based on the user query, the intent is:\n\nupd...",Update Memos,Update Memos
9,What appointments do I have scheduled for next...,Query Schedules,update Schedules,"Based on the user query, the intent can be inf...",Query Schedules.,Query Schedules


In [71]:
intent_list = ", ".join(intent_examples.intent.unique())

template = """You are a helpful assistant capable of detecting the intent behind a user's query.
The intent is a label that describes the user's purpose about taking action on the data in the database.

---
Explanation: 
'Schedules' are one-time events, 'RecurringSchedules' are events that repeat, and 'Memos' are something to be remembered without a specific time.
'Create' adds new entries, 'Delete' removes existing entries, 'Update' modifies existing entries, 'Query' retrieves existing entries without making any changes.

---
Example:
User Query (input): "Please remove my appointment from the schedule."
Intent (output): "Delete Schedules"
---

Reply ONE of the following: """+ intent_list + """
If no intent from the list is found, respond with: "Intent not found."

Provide a response to the following user query:
""" 

messages = [("system", template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

# check zero-shot prompt by intent_examples
answers = []
for i, row in intent_examples.iterrows():
    response = chain.invoke({"input": row["query"]})
    answers.append(response)
intent_examples["p5_answers"] = answers
intent_examples


,query,intent,predicted_output,fs_answers,p3_answers,p4_answers,p5_answers
0,Please create a new schedule for my dentist ap...,Create Schedules,create Schedule,"Based on the user query, I would determine the...",Create Schedules,Create Schedules,Delete Schedules
1,Set up a weekly team meeting every Monday at 1...,Create RecurringSchedules,The intent behind the user query is to create ...,"Based on the user query, I would classify it a...",Create RecurringSchedules,Create RecurringSchedules,Intent: Create RecurringSchedules
2,Add a note to remind me to buy groceries.,Create Memos,Create + Memos \n\n(Note: The user wants to cr...,"Based on the user query, I would determine the...",Create Memos,Create Memos,Intent: Create Memos
3,Remove my lunch appointment from the schedule ...,Delete Schedules,update Schedules \n\nThe query is asking to mo...,"Based on the user query, I would determine the...",Delete Schedules,Delete Schedules,Delete Schedules
4,Cancel my weekly yoga class scheduled for Thur...,Delete RecurringSchedules,update Schedules \n\n(assuming that cancelling...,"Based on the user query ""Cancel my weekly yoga...",Delete RecurringSchedules,Create RecurringSchedules \n\n(Note: The inten...,Intent: Delete RecurringSchedules
5,Delete the memo about calling my friend.,Delete Memos,update + Memos,"Based on the user query and possible intents, ...",Delete Memos,Delete Memos,Delete Memos.
6,Change my dentist appointment to November 12th...,Update Schedules,"To identify the intent behind the user query, ...","Based on the user query, I would determine the...",Create Schedules,Create Schedules,Delete Schedules
7,Update my weekly team meeting to start at 11 A...,Update RecurringSchedules,create Schedule \n\nThe intent is to modify th...,Intent: update RecurringSchedules,Create Schedules,Create Schedules \n\n(This operation involves ...,Delete Schedules
8,Edit my grocery list memo to include eggs and ...,Update Memos,"Based on the user query, I identify the intent...","Based on the user query, the intent is:\n\nupd...",Update Memos,Update Memos,Intent: Update Memos
9,What appointments do I have scheduled for next...,Query Schedules,update Schedules,"Based on the user query, the intent can be inf...",Query Schedules.,Query Schedules,Query RecurringSchedules


In [74]:
# few-shot prompt
fs_intent_template = """
Determine the operation type and table from the user queries below. Use only the following candidates: operation types (create, delete, update, query) and tables (Schedules, RecurringSchedules, Memos).
---
Example:

Query: 'Please remove my appointment from the schedule.'
Intent: delete Schedules
Query: 'Update the weekly team meeting to 11 AM.'
Intent: update RecurringSchedules
Query: 'Create a note for my presentation next week.'
Intent: create Memos
Query: 'Can you show my upcoming events?'
Intent: query Schedules

---
Reply ONE of the following: """+ intent_list + """
If no intent from the list is found, respond with: "Intent not found."
User Query: {input}
Intent:"

"""

messages = [("system", fs_intent_template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

# check zero-shot prompt by intent_examples
answers = []
for i, row in intent_examples.iterrows():
    response = chain.invoke({"input": row["query"]})
    answers.append(response)
intent_examples["p6_answers"] = answers
intent_examples


,query,intent,predicted_output,fs_answers,p3_answers,p4_answers,p5_answers,p6_answers
0,Please create a new schedule for my dentist ap...,Create Schedules,create Schedule,"Based on the user query, I would determine the...",Create Schedules,Create Schedules,Delete Schedules,Create Schedules
1,Set up a weekly team meeting every Monday at 1...,Create RecurringSchedules,"To identify the intent behind the user query, ...","Based on the user query, I would classify it a...",Create RecurringSchedules,Create RecurringSchedules,Intent: Create RecurringSchedules,Create RecurringSchedules
2,Add a note to remind me to buy groceries.,Create Memos,create Memos \n\nThe user wants to add a new r...,"Based on the user query, I would determine the...",Create Memos,Create Memos,Intent: Create Memos,Create Memos
3,Remove my lunch appointment from the schedule ...,Delete Schedules,Update \nSchedule,"Based on the user query, I would determine the...",Delete Schedules,Delete Schedules,Delete Schedules,Intent: delete Schedules
4,Cancel my weekly yoga class scheduled for Thur...,Delete RecurringSchedules,"Based on the user query, I would classify the ...","Based on the user query ""Cancel my weekly yoga...",Delete RecurringSchedules,Create RecurringSchedules \n\n(Note: The inten...,Intent: Delete RecurringSchedules,Delete RecurringSchedules
5,Delete the memo about calling my friend.,Delete Memos,"update + Memos \n\nHowever, since it's asking ...","Based on the user query and possible intents, ...",Delete Memos,Delete Memos,Delete Memos.,Delete Memos
6,Change my dentist appointment to November 12th...,Update Schedules,Create Schedule \n\nThe intent behind the user...,"Based on the user query, I would determine the...",Create Schedules,Create Schedules,Delete Schedules,Update Schedules \n\nExplanation:\nThe user is...
7,Update my weekly team meeting to start at 11 A...,Update RecurringSchedules,update Schedules,Intent: update RecurringSchedules,Create Schedules,Create Schedules \n\n(This operation involves ...,Delete Schedules,Update RecurringSchedules
8,Edit my grocery list memo to include eggs and ...,Update Memos,"Based on the user query, I identify the intent...","Based on the user query, the intent is:\n\nupd...",Update Memos,Update Memos,Intent: Update Memos,Update Memos
9,What appointments do I have scheduled for next...,Query Schedules,"Intent not found. \n\nHowever, based on the qu...","Based on the user query, the intent can be inf...",Query Schedules.,Query Schedules,Query RecurringSchedules,Query: What appointments do I have scheduled f...


In [76]:
intent_examples.to_excel("intent_examples_with_answers.xlsx", index=False)

## Step 2: Extract the entities from the user query

In [29]:
template = """You are a helpful assistant capable of extracting events from user queries.
    Identify the event in the following user queries.
    Output the event in the following format: "Event: {event}"
    ---
    For example,
    
    """
    
messages = [("system", template), ("human", "{input}")]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm

user_query = "remember to have a meeting tomorrow evening"

chain.invoke({"input":user_query})

'{"location/time": ["tomorrow evening"], "organization/event": ["meeting"]}'

In [12]:
from langchain.chains import create_sql_query_chain
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1")
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "When is the birthday party?"})
if "\nSQLQuery:" in response:
    response = response.split("\nSQLQuery:")[1]

response

" SELECT `id`, `event_date` FROM schedules WHERE `event` = 'Birthday Party' LIMIT 5"

In [8]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [13]:
db.run(response)

'[(2, datetime.date(2024, 2, 5))]'

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "When is the birthday party?"})

'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'Question: When is the birthday party?\\nSQLQuery: SELECT `event_date` FROM schedul\' at line 1")\n[SQL: Question: When is the birthday party?\nSQLQuery: SELECT `event_date` FROM schedules WHERE `event` = \'Birthday Party]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

query = full_chain.invoke(
    {
        "question": "When is the birthday party?"
    }
)
query

"SELECT event_date \nFROM schedules \nWHERE event = 'Birthday Party';"

In [6]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "When is the birthday party?"})

NameError: name 'write_query' is not defined

# Create Agents

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1")
agent_executor = create_sql_agent(llm, db=db, agent_type="zero-shot-react-description", verbose=True)

In [5]:
agent_executor.invoke(
    "When is the birthday party?"
)



> Entering new SQL Agent Executor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Thought: Let me first get a list of all the tables in the database.
Action: sql_db_list_tables
Action Input`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x116962020>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x116962020>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x116962020>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [14]:
from langchain_community.agent_toolkits import create_sql_agent
# from langchain_openai import ChatOpenAI

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)